<a href="https://colab.research.google.com/github/2JooYeon/2021_cau_oss_hackathon/blob/main/hackathon_template_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
*   모든 구현은 [2. 데이터 전처리] 및 [3.모델 생성]에서만 진행
*   [4. 모델 저장]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *    트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
 *  모델 발표 자료 
* 제출 기한: **오후 6시 (단, 발표자료는 12시)**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2021_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 두개의 테스트 데이터 셋 분류 정확도에 대한 weighted sum
 *  model.evaluate(x_test1, y_test1) + model.evaluate(x_test2, y_test2) * 2
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예: 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성
 *  단, tensorflow.data 및 dataset API를 사용하기 위해 변경하는 것은 허용. 이 경우, model evaluation 파트도 해당 API를 쓰도록 변경  
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [ ]:
# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras.utils import np_utils

# 데이터셋 로드 (Training dataset: CIFAR10, test dataset: CIFAR10 & CIFAR10의 변형)
(x_train, y_train), (x_test1, y_test1) = keras.datasets.cifar10.load_data()

test_ds2 = tfds.load('cifar10_1/v6', split='test', shuffle_files=False, batch_size=-1)
test_ds2 = tfds.as_numpy(test_ds2)
x_test2, y_test2 = test_ds2['image'], test_ds2['label']

#분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test1 = np_utils.to_categorical(y_test1)
y_test2 = np_utils.to_categorical(y_test2)

# 총 클래스 개수
num_classes = y_train.shape[1]
input_shape = x_train.shape[1:]

Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


# **2. 데이터 전처리**



In [2]:
# 데이터 전처리 (예: normalization)
# 원본 데이터와 전처리 후 데이터를 구분하기 위해, 변수명 x_train_after, x_test1_after, x_test2_after를 변경하지 말 것
x_train_after = x_train / 255.0
x_test1_after = x_test1 / 255.0
x_test2_after = x_test2 / 255.0

# **3. 모델 생성**

In [3]:
from keras import backend as K
from keras.layers import Layer

# shakeshake 레이어 클래스 
class ShakeShake(Layer):
    """ Shake-Shake-Image Layer """

    def __init__(self, **kwargs):
        super(ShakeShake, self).__init__(**kwargs)

    def build(self, input_shape):
        super(ShakeShake, self).build(input_shape)

    def call(self, x):
        assert isinstance(x, list)
        x1, x2 = x
        batch_size = K.shape(x1)[0]
        alpha = K.random_uniform((batch_size, 1, 1, 1))
        beta = K.random_uniform((batch_size, 1, 1, 1))
        # S-S-I 모델 
        def x_shake():
            return beta * x1 + (1 - beta) * x2 + K.stop_gradient((alpha - beta) * x1 + (beta - alpha) * x2)
        # E-E-I 모델 
        def x_even():
            return 0.5 * x1 + 0.5 * x2
        return K.in_train_phase(x_shake, x_even)

    def compute_output_shape(self, input_shape):
        assert isinstance(input_shape, list)
        return input_shape[0]


In [4]:
# L2 정규화 
l2 = keras.regularizers.l2(1e-4)

# ReLU -> Conv2D -> 배치 정규화 2번 반복 
def create_residual_branch(x, filters, stride):
    x = keras.layers.ReLU()(x)
    x = keras.layers.Conv2D(filters, kernel_size=3, strides=stride, padding='same',
                            kernel_initializer='he_normal', kernel_regularizer=l2,
                            use_bias=False)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = keras.layers.Conv2D(filters, kernel_size=3, strides=1, padding='same',
                            kernel_initializer='he_normal', kernel_regularizer=l2,
                            use_bias=False)(x)
    x = keras.layers.BatchNormalization()(x)
    return x


def create_residual_shortcut(x, filters, stride):
    x = keras.layers.ReLU()(x)
    x1 = keras.layers.Lambda(lambda y: y[:, 0:-1:stride, 0:-1:stride, :])(x)
    x1 = keras.layers.Conv2D(filters // 2, kernel_size=1, strides=1, padding='valid',
                             kernel_initializer='he_normal', kernel_regularizer=l2,
                             use_bias=False)(x1)
    x2 = keras.layers.Lambda(lambda y: y[:, 1::stride, 1::stride, :])(x)
    x2 = keras.layers.Conv2D(filters // 2, kernel_size=1, strides=1, padding='valid',
                             kernel_initializer='he_normal', kernel_regularizer=l2,
                             use_bias=False)(x2)
    x = keras.layers.Concatenate()([x1, x2])
    x = keras.layers.BatchNormalization()(x)
    return x


def create_residual_block(x, filters, stride=1):
    x1 = create_residual_branch(x, filters, stride)
    x2 = create_residual_branch(x, filters, stride)
    if stride > 1: x = create_residual_shortcut(x, filters, stride)
    return keras.layers.Add()([x, ShakeShake()([x1, x2])])


def create_residual_layer(x, filters, blocks, stride):
    x = create_residual_block(x, filters, stride)
    for i in range(1, blocks):
        x = create_residual_block(x, filters, 1)
    return x


def create_shakeshake_cifar(n_classes, n_blocks=[5, 5, 5], activation='softmax'):
    # Input and first convolutional layer
    x_in = keras.layers.Input(shape=(32, 32, 3))
    x = keras.layers.Conv2D(16, kernel_size=3, strides=1, padding='same',
                            kernel_initializer='he_normal', kernel_regularizer=l2,
                            use_bias=False)(x_in)
    x = keras.layers.BatchNormalization()(x)
    # Three stages of four residual blocks
    x = create_residual_layer(x, 16, n_blocks[0], 1)
    x = create_residual_layer(x, 32, n_blocks[1], 2)
    x = create_residual_layer(x, 64, n_blocks[2], 2)
    # Output pooling and dense layer
    x = keras.layers.ReLU()(x)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x_out = keras.layers.Dense(n_classes, activation=activation, kernel_initializer='he_normal')(x)
    return keras.models.Model(x_in, x_out)

In [ ]:
import keras
import numpy as np

from matplotlib import pyplot as plt
from keras.applications import resnet
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

epochs = 10
batch_size = 128

x_train_mean = np.mean(x_train_after, axis=0)
x_train_after -= x_train_mean
x_test1_after -= x_train_mean
x_test2_after -= x_train_mean
print(x_train_after.shape)
print(y_train.shape)


# Data augmentation
augment = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

# Learning rate 변화 주기
def schedule(epoch):
    if epoch >= 10: return 1e-5
    if epoch >= 5: return 1e-4
    return 1e-3
lr = LearningRateScheduler(schedule)


# cut-out함수 
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

# data-augmentation(cut-out 함수 적용)
img_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    zoom_range=0.1,
    fill_mode='nearest',
    preprocessing_function = get_random_eraser(v_l=0, v_h=255),
)

img_gen.fit(x_train_after, augment=True)

train_size = x_train_after.shape[0]
augment_size=100000
# x_train_after에서 random으로 뽑아와서 해당 데이터를 augmentation 진행
idx = np.random.randint(train_size, size=augment_size)
x_augmented = x_train_after[idx].copy()
y_augmented = y_train[idx].copy()
x_augmented = img_gen.flow(x_augmented, np.zeros(augment_size), batch_size=augment_size, shuffle=False).next()[0]

# 기존 데이터와 추가된 이미지 합치기
x_train_after = np.concatenate((x_train_after, x_augmented))
y_train = np.concatenate((y_train, y_augmented))

# 모델 컴파일
model = create_shakeshake_cifar(n_classes=10)
model.summary()
model.compile('Adam', 'categorical_crossentropy', metrics=['accuracy'])

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/checkpoint_entire_best2.h5', monitor='val_accuracy', verbose=1, save_weight_only=False, save_best_only=True, mode='auto')

# train
shake = model.fit(x_train_after, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test2_after, y_test2), callbacks=[lr, cp_callback])


(50000, 32, 32, 3)
(50000, 10)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 32, 32, 16)   0           batch_normalization[0][0]        
_______________________________________________________________

# **4. 모델 저장**

In [ ]:
save_path = '/content/'
team_name = 'team10'

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_'+ team_name + '.h5')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


# **5. 모델 로드 및 평가**

In [ ]:
from keras.models import load_model
# Assuming your model includes instance of an "AttentionLayer" class
save_path = '/content/'
team_name = 'team10'

# 커스텀 레이어 옵션 추가 
model = keras.models.load_model(save_path + 'model_entire_' + team_name + '.h5', custom_objects={'ShakeShake': ShakeShake})

model.evaluate(x_test1_after, y_test1)
model.evaluate(x_test2_after, y_test2)

63/63 [==============================] - 2s 26ms/step - loss: 0.9391 - accuracy: 0.7500


[0.9391130805015564, 0.75]